In [73]:
%matplotlib inline
# Importing libraries
import matplotlib.pyplot as plt
import pandas as pd

FILEPATH_PREFIX = '../../../datasets'
FILENAME = 'blackwells.csv'
FILEPATH = '{}/{}'.format(FILEPATH_PREFIX, FILENAME)
FILEPATH

'../../../datasets/blackwells.csv'

In [74]:
df = pd.read_csv(FILEPATH)

## Data analisys
Some preliminary analisys of the dataset

In [75]:
print("First 5 rows")
print("------------")
df.head()

First 5 rows
------------


,url,referer_url,src,shingle_vector
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 6, 3, 1)"
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 0, 3, 0)"
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 0, 1, 0, 0, 3, 1)"
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 1, 1, 1, 0, 0, 1)"
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c...","(2, 22, 1, 1, 7, 15, 7, 5)"


In [76]:
print("No. of rows and columns")
print("-----------------------")
df.shape

No. of rows and columns
-----------------------


(10919, 4)

In [77]:
print("Check null values")
print("-----------------")
df.isnull().any().any()

Check null values
-----------------


False

In [78]:
print("Check duplicate values")
print("----------------------")
len(df['url'].unique()) != df.shape[0]

Check duplicate values
----------------------


False

In [79]:
print("DataFrame column types")
print("----------------------")
df.info()

DataFrame column types
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10919 entries, 0 to 10918
Data columns (total 4 columns):
url               10919 non-null object
referer_url       10919 non-null object
src               10919 non-null object
shingle_vector    10919 non-null object
dtypes: object(4)
memory usage: 341.3+ KB


In [80]:
print("Some stats")
print("----------------")
df.describe()

Some stats
----------------


,url,referer_url,src,shingle_vector
count,10919,10919,10919,10919
unique,10919,6375,10525,73
top,https://blackwells.co.uk/bookshop/product/Smar...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 0, 1, 0, 3, 0)"
freq,1,12,7,2197


## Page labelling
Label (possibly) each page with its own template. Let's start with product (books) pages. 
The following URL links to a page which shows info regarding a book:

```
https://blackwells.co.uk/bookshop/product/Players-Handbook-by-Jeremy-Crawford-author-James-Wyatt-author-Robert-J-Schwalb-author-Bruce-R-Cordell-author/9780786965601
```

So we could infer that each product page has the following url schema:

```
https://blackwells.co.uk/bookshop/product/
```

In [81]:
import re

#Urls associated with product pages
testStrings1= ['https://blackwells.co.uk/bookshop/product/Dungeons--Dragons-Starter-Set-Six-Dice-Five-Ready-to-Play-DD-Characters-With-Character-Sheets-a-Rulebook-and-One-Adventure-by-Wizards-RPG-Team-author/9780786965595',
              'https://blackwells.co.uk/bookshop/product/Education-by-Gary-Thomas-author/9780199643264',
              'https://blackwells.co.uk/bookshop/product/Game-of-Thrones-House-Stark-Ruled-Notebook-Reformat-by-Insight-Editions-author/9781683836650',
              'https://blackwells.co.uk/bookshop/product/The-World-of-Peter-Rabbit-by-Beatrix-Potter-author/9780723275923',
              'https://blackwells.co.uk/bookshop/product/Happy-Old-Me-by-DAVIES-HUNTER/9781472626028',
              'https://blackwells.co.uk/bookshop/product/Hey-Warrior-by-Karen-Young-Norvile-Dovidonyte-artist/9781912678037']

#Various urls which aren't product pages
testStrings2 = ['https://blackwells.co.uk/bookshop/bestsellers',
               'https://blackwells.co.uk/rarebooks/book.php?id=55981',
               'https://blackwells.co.uk/bookshop/category/_computing/',
               'https://blackwells.co.uk/bookshop/search/author/Subrata%20Dasgupta%20(author)',
               'https://blackwells.co.uk/bookshop/search/?productTitle=game+of+thrones&quantity=6&author=&quantity=6&publisher=&quantity=6&keyword=&quantity=6&pubDateFrom=&pubDateTo=&isbn=&priceFrom=&priceTo=&academicLevel=E',
               'https://blackwells.co.uk/bookshop/category/WN/',
               'https://blackwells.co.uk/bookshop/category/WQ/',
               'https://blackwells.co.uk/bookshop/search/?keyword=',
               'https://blackwells.co.uk/bookshop/basket',
               'https://blackwells.co.uk/bookshop/events/',
               'https://blackwells.co.uk/bookshop/collection/Gift-and-Stationery-bestsellers/-186139',
               'https://blackwells.co.uk/bookshop/search/?keyword=game+of+thrones',
               'https://blackwells.co.uk/bookshop/category/_travel/']

testStrings = testStrings1 + testStrings2

productRegex = r'^https\:\/\/blackwells\.co\.uk\/bookshop\/product\/.+$'

# simple tests
print("Testing article regex:")
print("Only the first 6 urls should be matching with the article regex")
print("---------------------------------------------------------------")

for uri in testStrings:
    print(re.match(productRegex, uri))
    
print('---------------------------------------------------------------')

Testing article regex:
Only the first 6 urls should be matching with the article regex
---------------------------------------------------------------
<re.Match object; span=(0, 205), match='https://blackwells.co.uk/bookshop/product/Dungeon>
<re.Match object; span=(0, 87), match='https://blackwells.co.uk/bookshop/product/Educati>
<re.Match object; span=(0, 134), match='https://blackwells.co.uk/bookshop/product/Game-of>
<re.Match object; span=(0, 106), match='https://blackwells.co.uk/bookshop/product/The-Wor>
<re.Match object; span=(0, 85), match='https://blackwells.co.uk/bookshop/product/Happy-O>
<re.Match object; span=(0, 108), match='https://blackwells.co.uk/bookshop/product/Hey-War>
None
None
None
None
None
None
None
None
None
None
None
None
None
---------------------------------------------------------------


Next we have to label pages which show list of books. We noted that books in blackwells.co.uk are grouped by category, so that books which share the same category are listed in the same page in a paginated way. 

```
https://blackwells.co.uk/bookshop/category/_travel/
```

So urls referring to categories (seems to) have the following schema:

```
https://blackwells.co.uk/bookshop/category/<somethingElse>
```

Note that also search results (except `https://blackwells.co.uk/bookshop/search/?keyword=` and `https://blackwells.co.uk/bookshop/search/`) are paginated:

```
https://blackwells.co.uk/bookshop/search/?keyword=game+of+thrones
```

In that case the url schema follows this pattern:
```
https://blackwells.co.uk/bookshop/search/<?keyword=.+ or .+>
```

In [82]:
catalogRegex = r'^https\:\/\/blackwells\.co\.uk\/bookshop\/(category\/.+|(search\?.+|search\/\?keyword\=.+|search\/(?!(\?keyword\=))))'

print("Testing catalog regex:")
print("Only 7 urls should be matching with the bestsellers regex")
print("-----------------------------------------------------------")

for uri in testStrings:
    print(re.match(catalogRegex, uri))
print('---------------------------------------------------------------')

Testing catalog regex:
Only 7 urls should be matching with the bestsellers regex
-----------------------------------------------------------
None
None
None
None
None
None
None
None
<re.Match object; span=(0, 54), match='https://blackwells.co.uk/bookshop/category/_compu>
<re.Match object; span=(0, 41), match='https://blackwells.co.uk/bookshop/search/'>
<re.Match object; span=(0, 41), match='https://blackwells.co.uk/bookshop/search/'>
<re.Match object; span=(0, 46), match='https://blackwells.co.uk/bookshop/category/WN/'>
<re.Match object; span=(0, 46), match='https://blackwells.co.uk/bookshop/category/WQ/'>
None
None
None
None
<re.Match object; span=(0, 65), match='https://blackwells.co.uk/bookshop/search/?keyword>
<re.Match object; span=(0, 51), match='https://blackwells.co.uk/bookshop/category/_trave>
---------------------------------------------------------------


## Add `label` column
Labelling each possible row of the dataframe

In [83]:
df['label'] = None
df.head()

,url,referer_url,src,shingle_vector,label
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 6, 3, 1)",None
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 0, 3, 0)",None
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 0, 1, 0, 0, 3, 1)",None
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 1, 1, 1, 0, 0, 1)",None
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c...","(2, 22, 1, 1, 7, 15, 7, 5)",None


In [84]:
def set_label(url, regex, old_label, new_label):
    if re.match(regex, url):
        if old_label is None:
            return new_label
        else:
            return 'collision'
    else:
        return old_label

Add product label

In [85]:
df['label'] = df.apply(lambda x: set_label(x['url'], productRegex, x['label'], 'product'), axis=1)
print("There should be no row with 'collision' label")
print("---------------------------------------------")
df[df['label']=='collision'].head()

There should be no row with 'collision' label
---------------------------------------------


,url,referer_url,src,shingle_vector,label


Add list label for paginated pages

In [86]:
df['label'] = df.apply(lambda x: set_label(x['url'], catalogRegex, x['label'], 'list'), axis=1)
print("There should be no row with 'collision' label")
print("---------------------------------------------")
df[df['label']=='collision'].head()

There should be no row with 'collision' label
---------------------------------------------


,url,referer_url,src,shingle_vector,label


Summary:

In [87]:
fmt_string = 'There are {} row with {} label'
print(fmt_string.format(len(df[df['label'].isnull()]),'no'))
print(fmt_string.format(len(df[df['label']=='product']), 'product'))
print(fmt_string.format(len(df[df['label']=='list']), 'list'))

There are 20 row with no label
There are 10405 row with product label
There are 494 row with list label


In [90]:
df[df['label'].isnull()][['url', 'referer_url']].head(20)

,url,referer_url
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket
5,https://blackwells.co.uk/bookshop/delivery,https://blackwells.co.uk/bookshop/basket
6,https://blackwells.co.uk/bookshop/editorial/wi...,https://blackwells.co.uk/bookshop/home
7,https://blackwells.co.uk/bookshop/editorial/co...,https://blackwells.co.uk/bookshop/basket
8,https://blackwells.co.uk/bookshop/editorial/bu...,https://blackwells.co.uk/bookshop/basket
9,https://blackwells.co.uk/bookshop/editorial/se...,https://blackwells.co.uk/bookshop/basket
10,https://blackwells.co.uk/bookshop/about,https://blackwells.co.uk/bookshop/basket
11,https://blackwells.co.uk/bookshop/readinglists...,https://blackwells.co.uk/bookshop/home


Note that there are some rows which aren't labeled. That is because these pages aren't generated neither by the 'product' template nor the 'list' template.

## Write dataset to a file

In [91]:
df.to_csv(FILEPATH, encoding='utf-8', index=False)

In [92]:
test = pd.read_csv(FILEPATH)

In [93]:
test.head()

,url,referer_url,src,shingle_vector,label
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 6, 3, 1)",NaN
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 0, 3, 0)",list
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 0, 1, 0, 0, 3, 1)",NaN
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 1, 1, 1, 0, 0, 1)",product
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c...","(2, 22, 1, 1, 7, 15, 7, 5)",NaN


In [94]:
test.shape

(10919, 5)

In [95]:
print(fmt_string.format(len(test[test['label'].isnull()]),'no'))
print(fmt_string.format(len(test[test['label']=='product']), 'product'))
print(fmt_string.format(len(df[df['label']=='list']), 'list'))

There are 20 row with no label
There are 10405 row with product label
There are 494 row with list label
